Import the relevant librries

In [40]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import pickle
import time, datetime, os
%reload_ext tensorboard

Set tensorboard and import final_dataset as stats.


In [32]:
NAME = "264x32x16x1_MSE_sv{}".format(int(time.time()))
log_dir = "logs/fit/" + NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
stats = pd.read_csv(r'Data\final_dataset_nonzero.csv')
#stats = stats.drop('Score',1)


Slice stats into X and y. Split the data into training and test set (80/20) and normalize the data

In [33]:
X = stats.iloc[:,3:-4]
y = stats.iloc[:,-1:]
random_state = 12
#split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, random_state=random_state)
X_train = tf.keras.utils.normalize(X_train, axis=1)
X_test = tf.keras.utils.normalize(X_test, axis=1)

Define the custom loss functions

BCE: Binary-cross entropy with added parameters to optimize our return on investment.

$L(y\hat, y) = -\sum\limits_{i=1}^{m} (Odds_{1i}*y_{i}-1) * log(Odds_{1i}*\hat y_{i}-1) + (Odds_{2i}(1 - y_{i})-1) * log(Odds_{2i}*(1- \hat y_{i}) -1)$

Source: https://www.vantage-ai.com/en/blog/beating-the-bookies-with-machine-learning

MSE: Mean Squared Error with odds decorrelation

$L(p\hat, y)= \frac{1}{N} \sum\limits_{i=1}^{N}(\hat p_{i} - y_{i})^{2} - C * (\hat p_{i} - \frac {1}{Odds_{i}})^{2}$

The first part $(\hat p_{i} - y_{i})^{2}$ is the Mean Squared Error, the difference between the predicted outcome versus the actual aoutcome. For the second part, C is a constant that determines the significance of the decorrelation effect. $(\hat p_{i} - \frac {1}{Odds_{i}})^{2}$, $\hat p_{i}$ is the predicted probability for the home team and $\frac {1}{Odds_{i}}$ is the probability that the bookmakers gives to the home team victory.

Source: Hubáček, Ondřej, Gustav Šourek, and Filip Železný. "Exploiting sports-betting market using machine learning." International Journal of Forecasting 35.2 (2019): 783-796.


In [35]:
from tensorflow.python.ops import math_ops

def coef_bce(y_pred,y_true, odds1, odds2):
    return -1 * K.mean((odds1 * y_true -1) * K.log(odds1 * y_pred -1) + (odds2 * (1- y_true) -1) * K.log(odds2 * (1-y_pred)-1))

def bce_custom(odds1, odds2):
    def bce(y_pred, y_true):
        return coef_bce(y_pred, y_true, odds1, odds2)
    return bce

#loss_bce = bce_custom(odds1, odds2)


def coef_mse(odds_game, y_true, y_pred):
    mse = math_ops.squared_difference(y_pred, y_true)  #squared difference
    odds = math_ops.squared_difference(y_pred, odds_game)   
    loss = K.mean(mse - 0.05 * odds, axis=-1) #mean over last dimension
    return loss

def mse_custom(odds_game):
    def mse(y_true, y_pred):
        return coef_mse(y_pred, y_true, odds_game )
    return mse

#loss_mse = mse_custom(odds1)


def mse_simple(y_true, y_pred):
    mse = math_ops.squared_difference(y_pred, y_true) 
    loss = K.mean(mse, axis=-1)
    return loss
    

## Model Architecture:
After configuring GridSearch, the best parameters where:
- layers: 32x32 // Input is 264(n. of features) and output is 1. So (264 x 32 x 32 x 1)
- epochs: 5
- batch size: 3
-

In [36]:
#opt = tf.keras.optimizers.Adam(clipnorm=0.3)

model = Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(Dense(264, input_dim=264, kernel_initializer='normal', activation='relu', bias_regularizer=l2(0.01)))
model.add(Dense(32, kernel_initializer='normal', activation='relu', bias_regularizer=l2(0.01)))
model.add(Dense(32, kernel_initializer='normal', activation='relu', bias_regularizer=l2(0.01)))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model.compile(optimizer='Adam', loss='MeanSquaredError', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size= 3, epochs=5, validation_data=(X_test, y_test),callbacks=[tensorboard_callback])

Epoch 1/5
3800/3800 [==============================] - 3s 834us/step - loss: 0.2209 - accuracy: 0.6486 - val_loss: 0.2161 - val_accuracy: 0.6636
Epoch 2/5
3800/3800 [==============================] - 3s 737us/step - loss: 0.2167 - accuracy: 0.6582 - val_loss: 0.2166 - val_accuracy: 0.6608
Epoch 3/5
3800/3800 [==============================] - 3s 731us/step - loss: 0.2144 - accuracy: 0.6648 - val_loss: 0.2155 - val_accuracy: 0.6615
Epoch 4/5
3800/3800 [==============================] - 3s 723us/step - loss: 0.2128 - accuracy: 0.6668 - val_loss: 0.2163 - val_accuracy: 0.6566
Epoch 5/5
3800/3800 [==============================] - 3s 708us/step - loss: 0.2111 - accuracy: 0.6696 - val_loss: 0.2151 - val_accuracy: 0.6650


In [37]:
val_loss, val_acc = model.evaluate(X_test, y_test)


90/90 [==============================] - 0s 540us/step - loss: 0.2151 - accuracy: 0.6650


In [38]:
prediction = model.predict(X_test[:])
label = y_test[:]
prediction_l = prediction.tolist()
pred =pd.DataFrame(prediction_l, columns=['prediction'])

In [39]:
stats.reset_index
index = stats[['GAME_DATE', 'HOME', 'AWAY','Odds1', 'Odds2', 'Score']]
data = label.join(index)
data.reset_index(inplace=True)
pred_data = data.join(pred)
pred_data.to_csv('predicted_data.csv')
label

,Results
8985,1
7320,1
1662,0
2694,0
3665,1
...,...
3446,0
10005,1
4325,0
5980,1


In [43]:
#%tensorboard --logdir logs/fit

In [7]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
def model_nn(units='8', drop='0,1'):
    model = Sequential()
    #model.add(tf.keras.layers.Flatten())
    model.add(Dense(264, input_dim=264, kernel_initializer='normal', activation='relu',bias_regularizer=l2(0.01)))
    model.add(Dense(units=units, kernel_initializer='normal', activation='relu',bias_regularizer=l2(0.01)))
    model.add(Dense(units=units, kernel_initializer='normal', activation='relu',bias_regularizer=l2(0.01)))
    model.add(Dropout(drop))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

    model.compile(optimizer='adam', loss='MeanSquaredError', metrics=['accuracy'])
    #model.fit(X_train, y_train, batch_size= 5, epochs=10, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    return model

model = KerasClassifier(build_fn=model_nn)
params={#'bias':[0.01, 0.03, 0.05, 0.07, 0.1], 
        'drop':[0.0, 0.1, 0.2, 0.4, 0.5],
        'units':[8,16, 32, 64],
        'batch_size':[3], 
        'nb_epoch':[5]

        }
gs=GridSearchCV(estimator=model, param_grid=params, cv=10)
# now fit the dataset to the GridSearchCV object. 
gs = gs.fit(X_train, y_train)

3432/3432 [==============================] - 2s 684us/step - loss: 0.2216 - accuracy: 0.6447


3432/3432 [==============================] - 2s 574us/step - loss: 0.2271 - accuracy: 0.6423


3432/3432 [==============================] - 2s 562us/step - loss: 0.2262 - accuracy: 0.6370


382/382 [==============================] - 0s 374us/step - loss: 0.2149 - accuracy: 0.6687


3432/3432 [==============================] - 2s 652us/step - loss: 0.2254 - accuracy: 0.6400


3814/3814 [==============================] - 2s 621us/step - loss: 0.2249 - accuracy: 0.6410


In [10]:
best_params=gs.best_params_
accuracy=gs.best_score_
accuracy
best_params

{'batch_size': 3, 'drop': 0.4, 'nb_epoch': 5, 'units': 16}